In [1]:
import numpy as np
import scipy.sparse as sp
import os
import random
from copy import deepcopy

In [2]:
import pandas as pd

In [3]:
os.getcwd()
os.chdir('/home/dqw_zyf/GraphDTI')

In [4]:
import    pyximport

pyximport.install()

from graphsaint.globals import *
from graphsaint.pytorch_version.models import GraphSAINT
from graphsaint.pytorch_version.minibatch import Minibatch
from graphsaint.utils import *
from graphsaint.metric import *
from graphsaint.pytorch_version.utils import *
from graphsaint.pytorch_version.train import *


import torch
import time

import warnings
warnings.filterwarnings("ignore")

fatal: Not a git repository (or any parent up to mount point /home)
Stopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).
fatal: Not a git repository (or any parent up to mount point /home)
Stopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).


In [5]:
os.chdir('/home/dqw_zyf/GraphDTI/data')

In [6]:
alldata = pd.read_csv('alldata.csv') #491718
tar_embeddings = pd.read_csv('corpus2_tar_embeddings.csv')
mol_embeddings = pd.read_csv('corpus2_mol_embeddings.csv')

alldata.drop_duplicates(inplace = True)
alldata = alldata[(alldata.notna())] #462361

In [7]:
data = pd.merge(alldata,tar_embeddings)
print(len(data))
data1 = pd.merge(data,mol_embeddings)
print(len(data1))
data1.drop_duplicates(inplace = True)
data1 = data1[(data1.notna())] #462361
print(len(data1))

462361
462361
462361


In [8]:
# 进行训练、验证和测试集划分
train = data1[data1['Curation/DataSource']=='ChEMBL']##399553
val = data1[data1['Curation/DataSource']=='Curated from the literature by BindingDB'] ##27899
test =  data1[data1['Curation/DataSource']=='PubChem'] ##34909


In [9]:
train_label = np.array(train['labels'])
test_BindingDB_label =  np.array(val['labels'])
graph_1 = pd.concat([train,val],axis=0)
graph_1 = graph_1.drop('mol_id',axis=1)
feats_1 = np.array(graph_1.iloc[:,9:209])

In [10]:
def data_prepare_modified(train_label,test_label,task,graph,feats,seed):
   
    assert task in  ['ChEMBL','BindingDB','PubChem']
    print('task:{}!!!!'.format(task))
    
    train_label = list(train_label)
    #train_label.extend(list(test_label))
    #class_map = deepcopy(train_label) #label
    class_map = dict(zip(range(len(graph)),list(graph['labels'])))
    
    #train_datapair = list(train_datapair)
    print('训练集数量：',len(train_label))
    
    train_label.extend(list(test_label))
    datapair = deepcopy(train_label)
    print("测试集数量：",len(test_label))
    
    
    
    
    '''-----------------------------------生成role--------------------------------------''' 
    
    tr_num = len(train_label)
    print(tr_num)
    random_index = list(range(tr_num))
    
    random.seed(seed)
    random.shuffle(random_index)
    
    
    k_folds = 5
    CV_size = int(tr_num / k_folds)
    #print(CV_size )
    temp = np.array(random_index[:tr_num - tr_num %k_folds]).reshape(k_folds, CV_size,  -1).tolist()
    #print(type(temp))
    #temp[k_folds - 1] = temp[k_folds - 1] + random_index[tr_num - tr_num % k_folds:]
    #print(type(temp[1],))
    tr_index = list(np.array(temp[1:]).reshape(1,CV_size*4))
    #tr_index = temp[1]
    #tr_index.extend(temp[2])
    #tr_index.extend(temp[3])
    #tr_index.extend(temp[4])
    #print(tr_index[0])
    val_index = list(np.array(temp[0]).reshape(1,CV_size))
    #print(type(val_index[0]))
    
    te_index = list(range(len(graph)-len(test_label),len(graph))) #list of all test node indices
    print(type(te_index))
    print(type(tr_index[0]))
    print(type(val_index[0]))
    print('tr:',len(tr_index[0]),'va',len(val_index[0]),'te',len(te_index))
    
    role ={}
    role['te'] = te_index
    role['va'] = val_index[0]
    role['tr'] = tr_index[0]
    
    """---------------------------生成邻接矩阵-------------------------------------------"""
    #adj_full = sp.load_npz(task_neighbors_edge)
    row = np.array(range(len(graph)))
    col = row
    data = np.ones(len(graph))
    adj_full = sp.csr_matrix((data, (row, col)),shape=(len(graph),len(graph)))
    
    
    #print(max(val_index[0]))
    index = np.hstack((val_index[0],te_index)) 
    #print(index.shape)
    #print(max(index))
    #print(len(index))
    train_row = list(range(len(graph)))
    #print(len(train_row))
    for idx in list(index):
        #print(idx)
        train_row[idx] = 0
    train_row =  list(filter(lambda x : x != 0, train_row))
    train_col = train_row
    train_data = np.ones(len(train_col))
    adj_train = sp.csr_matrix((train_data, (train_row, train_col)),shape=(len(graph),len(graph)))
    #adj_train = sp.csr_matrix(adj_train)  
    
    print("全部link的数量：",adj_full.sum(),"  训练link的数量： ",adj_train.sum())
    
    #feats = np.load(feats)
    print ('features:',type(feats),feats.shape) 
    
    return adj_full,adj_train,role,class_map,feats


In [11]:
def process_graph_data_modified(adj_full,adj_train,role,class_map,feats):
    """
    setup vertex property map for output classes, train/val/test masks, and feats
    """
    num_vertices = adj_full.shape[0]
    if isinstance(list(class_map.values())[0],list):
        num_classes = len(list(class_map.values())[0])
        class_arr = np.zeros((num_vertices, num_classes))
        for k,v in class_map.items():
            class_arr[k] = v
    else:
        num_classes = max(class_map.values()) - min(class_map.values()) + 1
        class_arr = np.zeros((num_vertices, num_classes))
        offset = min(class_map.values())
        for k,v in class_map.items():
            class_arr[k][v-offset] = 1
    return adj_full, adj_train, feats, class_arr, role

In [12]:
def prepare_1(train_data,train_params,arch_gcn):
    """
    Prepare some data structure and initialize model / minibatch handler before
    the actual iterative training taking place.
    """
    adj_full, adj_train, feat_full, class_arr,role = train_data
    adj_full = adj_full.astype(np.int32)
    adj_train = adj_train.astype(np.int32)
    adj_full_norm = adj_norm(adj_full)
    num_classes = class_arr.shape[1]

    minibatch = Minibatch(adj_full_norm, adj_train, role, train_params)
    model = GraphSAINT(num_classes, arch_gcn, train_params, feat_full, class_arr)
    printf("TOTAL NUM OF PARAMS = {}".format(sum(p.numel() for p in model.parameters())), style="yellow")
    minibatch_eval=Minibatch(adj_full_norm, adj_train, role, train_params, cpu_eval=True)
    model_eval=GraphSAINT(num_classes, arch_gcn, train_params, feat_full, class_arr, cpu_eval=True)
    #if args_global.gpu >= 0:
        #model = model.cuda()
    return model, minibatch, minibatch_eval, model_eval

In [13]:
def train_1(train_phases, model, minibatch, minibatch_eval, model_eval, eval_val_every):
    if not args_global.cpu_eval:
        minibatch_eval=minibatch
    epoch_ph_start = 0
    
    auc_best, ep_best = 0, -1
    time_train = 0
    dir_saver = '{}/pytorch_models'.format(args_global.dir_log)
    path_saver = '{}/pytorch_models/DTI_saved_model_{}.pkl'.format(args_global.dir_log, timestamp)
    print('dir_saver: ', dir_saver)
    print('path_saver: ', path_saver)
    for ip, phase in enumerate(train_phases):
        printf('START PHASE {:4d}'.format(ip),style='underline')
        minibatch.set_sampler(phase)
        num_batches = minibatch.num_training_batches()
        for e in range(epoch_ph_start, int(phase['end'])):
            printf('Epoch {:4d}'.format(e),style='bold')
            minibatch.shuffle()
            
            l_loss_tr, lr_accuracy_tr, lr_precision_tr, lr_recall_tr, lr_f1_tr, lr_roc_auc_tr, lr_aupr_tr, lr_pos_acc_tr, lr_neg_acc_tr = [], [], [], [], [], [], [], [], []
            
            time_train_ep = 0
            
            t1 = time.time()
            loss_train,preds_train,labels_train = model.train_step(*minibatch.one_batch(mode='train'))
            #print("loss_train:",loss_train)
            #print('preds_train:',preds_train.shape)
            #print('labels_train:',labels_train.shape)
            time_train_ep += time.time() - t1
            #if not minibatch.batch_num % args_global.eval_train_every:

            ys_train, metrics_train = metrics(to_numpy(labels_train[:, 1]),to_numpy(preds_train[:, 1]),model.sigmoid_loss, isprint = True)
            l_loss_tr.append(loss_train)
            lr_accuracy_tr.append(metrics_train[0])
            lr_precision_tr.append(metrics_train[1])
            lr_recall_tr.append(metrics_train[2])
            lr_f1_tr.append(metrics_train[3])
            lr_roc_auc_tr.append(metrics_train[4])
            lr_aupr_tr.append(metrics_train[5])
            lr_pos_acc_tr.append(metrics_train[6])
            lr_neg_acc_tr.append(metrics_train[7])
                    
            if (e+1)%eval_val_every == 0:
                if args_global.cpu_eval:
                    torch.save(model.state_dict(),'tmp.pkl')
                    model_eval.load_state_dict(torch.load('tmp.pkl',map_location=lambda storage, loc: storage))
                else:
                    model_eval = model
                    
                #printf('Train (Ep avg): loss = {:.4f} | Time = {:.4f}sec'.format(f_mean(l_loss_tr), time_train_ep), style = 'yellow')
                printf('acc={:.4f}|precision={:.4f}|recall={:.4f}|f1={:.4f}|auc={:.4f}|aupr={:.4f}|pos_acc={:.4f}|neg_acc={:.4f}'.format(
f_mean(lr_accuracy_tr), f_mean(lr_precision_tr), f_mean(lr_recall_tr), f_mean(lr_f1_tr), f_mean(lr_roc_auc_tr), f_mean(lr_aupr_tr), f_mean(lr_pos_acc_tr), f_mean(lr_neg_acc_tr)), 
style = 'yellow')
                              
                loss_val, ys_val, metrics_val = evaluate_full_batch(model_eval, minibatch_eval, mode='val')

                auc_val = metrics_val[4]
                if auc_val > auc_best:
                    auc_best, ep_best = auc_val, e
                    if not os.path.exists(dir_saver):
                        os.makedirs(dir_saver)
                    printf('  Saving model ...', style='yellow')
                    torch.save(model.state_dict(), path_saver)
            time_train += time_train_ep
        epoch_ph_start = int(phase['end'])
    printf("Optimization Finished!", style="yellow")
    if ep_best >= 0:
        if args_global.cpu_eval:
            model_eval.load_state_dict(torch.load(path_saver, map_location=lambda storage, loc: storage))
        else:
            model.load_state_dict(torch.load(path_saver))
            model_eval=model
        printf('  Restoring model ...', style='yellow')
    printf('Best Epoch = ' + str(ep_best), style = 'red')
    loss_test, ys_test, metrics_test = evaluate_full_batch(model_eval, minibatch_eval, mode='test')
    
    printf("Total training time: {:6.2f} sec".format(time_train), style='red')


In [15]:
"arch 1-0-1-0",'epoch 1000'
if __name__ == '__main__':
    print ('-----------------------------------this is train task--------------------------------')
    log_dir(args_global.train_config, args_global.data_prefix, git_branch, git_rev, timestamp)
    train_params, train_phases, arch_gcn = parse_n_prepare(args_global)
    print(train_params,train_phases,arch_gcn)
    tmp_data = data_prepare_modified(train_label,test_BindingDB_label, 'BindingDB',graph_1,feats_1,0)
    train_data = process_graph_data(*tmp_data)
    if 'eval_val_every' not in train_params:
        train_params['eval_val_every'] = 100
    model, minibatch, minibatch_eval, model_eval = prepare_1(train_data, train_params, arch_gcn)          
    train_1(train_phases, model, minibatch, minibatch_eval, model_eval, train_params['eval_val_every'])

-----------------------------------this is train task--------------------------------
{'lr': 0.001, 'weight_decay': 0.0, 'norm_loss': True, 'norm_aggr': True, 'q_threshold': 50, 'q_offset': 0, 'dropout': 0.1, 'sample_coverage': 50, 'positive_weight': 10} [{'end': 1000, 'sampler': 'edge', 'size_subg_edge': 2000, 'num_root': 750, 'depth': 4}] {'dim': 512, 'aggr': 'concat', 'loss': 'sigmoid', 'arch': '1-0-1-0', 'act': 'relu', 'bias': 'norm'}
task:BindingDB!!!!
训练集数量： 399553
测试集数量： 27899
427452
<class 'list'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
tr: 341960 va 85490 te 27899
全部link的数量： 427452.0   训练link的数量：  319430.0
features: <class 'numpy.ndarray'> (427452, 200)
TOTAL NUM OF PARAMS = 1787910
dir_saver:  ./pytorch_models
path_saver:  ./pytorch_models/DTI_saved_model_2023-11-10 16-34-57.pkl
START PHASE    0
sampling 200 subgraphs:   time = 0.217 sec
0.0
0.0
Epoch    0
tn = 159224, fp = 17072, fn = 227871, tp = 23285
y_pred: 0 = 387095 | 1 = 40357
y_true: 0 = 176296 | 1 = 251156
a

NameError: name 'cd' is not defined

In [25]:
train_label = np.array(train['labels'])
test_PubChem_label =  np.array(test['labels'])
graph_2 = pd.concat([train,test],axis=0)
graph_2 = graph_2.drop('mol_id',axis=1)
feats_2 = np.array(graph_2.iloc[:,9:209])

In [26]:
"arch 1-0-1-0",'epoch 1000'
if __name__ == '__main__':
    print ('-----------------------------------this is train task--------------------------------')
    log_dir(args_global.train_config, args_global.data_prefix, git_branch, git_rev, timestamp)
    train_params, train_phases, arch_gcn = parse_n_prepare(args_global)
    print(train_params,train_phases,arch_gcn)
    tmp_data = data_prepare_modified(train_label,test_PubChem_label, 'PubChem',graph_2,feats_2,2023)
    train_data = process_graph_data(*tmp_data)
    if 'eval_val_every' not in train_params:
        train_params['eval_val_every'] = 100
    model, minibatch, minibatch_eval, model_eval = prepare_1(train_data, train_params, arch_gcn)          
    train_1(train_phases, model, minibatch, minibatch_eval, model_eval, train_params['eval_val_every'])

-----------------------------------this is train task--------------------------------
{'lr': 0.001, 'weight_decay': 0.0, 'norm_loss': True, 'norm_aggr': True, 'q_threshold': 50, 'q_offset': 0, 'dropout': 0.1, 'sample_coverage': 50, 'positive_weight': 10} [{'end': 1000, 'sampler': 'edge', 'size_subg_edge': 2000, 'num_root': 750, 'depth': 4}] {'dim': 512, 'aggr': 'concat', 'loss': 'sigmoid', 'arch': '1-0-1-0', 'act': 'relu', 'bias': 'norm'}
task:PubChem!!!!
训练集数量： 399553
测试集数量： 34909
434462
<class 'list'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
tr: 347568 va 86892 te 34909
全部link的数量： 434462.0   训练link的数量：  319760.0
features: <class 'numpy.ndarray'> (434462, 200)
TOTAL NUM OF PARAMS = 1787910
dir_saver:  ./pytorch_models
path_saver:  ./pytorch_models/DTI_saved_model_2023-11-01 22-57-07.pkl
START PHASE    0
sampling 200 subgraphs:   time = 0.199 sec
Epoch    0
tn = 47889, fp = 148891, fn = 54548, tp = 183134
y_pred: 0 = 102437 | 1 = 332025
y_true: 0 = 196780 | 1 = 237682
acc=0.5317

In [24]:
"arch 1-1-0",'epoch 1000'
if __name__ == '__main__':
    print ('-----------------------------------this is train task--------------------------------')
    log_dir(args_global.train_config, args_global.data_prefix, git_branch, git_rev, timestamp)
    train_params, train_phases, arch_gcn = parse_n_prepare(args_global)
    print(train_params,train_phases,arch_gcn)
    tmp_data = data_prepare_modified(train_label,test_BindingDB_label, 'BindingDB',graph_1,feats_1,10)
    train_data = process_graph_data(*tmp_data)
    if 'eval_val_every' not in train_params:
        train_params['eval_val_every'] = 100
    model, minibatch, minibatch_eval, model_eval = prepare_1(train_data, train_params, arch_gcn)          
    train_1(train_phases, model, minibatch, minibatch_eval, model_eval, train_params['eval_val_every'])

-----------------------------------this is train task--------------------------------
{'lr': 0.001, 'weight_decay': 0.0, 'norm_loss': True, 'norm_aggr': True, 'q_threshold': 50, 'q_offset': 0, 'dropout': 0.1, 'sample_coverage': 50, 'positive_weight': 10} [{'end': 1000, 'sampler': 'edge', 'size_subg_edge': 2000, 'num_root': 750, 'depth': 4}] {'dim': 512, 'aggr': 'concat', 'loss': 'sigmoid', 'arch': '1-1-0', 'act': 'relu', 'bias': 'norm'}
task:BindingDB!!!!
训练集数量： 399553
测试集数量： 27899
427452
<class 'list'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
tr: 341960 va 85490 te 27899
全部link的数量： 427452.0   训练link的数量：  319745.0
features: <class 'numpy.ndarray'> (427452, 200)
TOTAL NUM OF PARAMS = 1786374
dir_saver:  ./pytorch_models
path_saver:  ./pytorch_models/DTI_saved_model_2023-11-11 18-22-53.pkl
START PHASE    0
sampling 200 subgraphs:   time = 0.299 sec
0.0
0.0
Epoch    0
tn = 141156, fp = 35140, fn = 195581, tp = 55575
y_pred: 0 = 336737 | 1 = 90715
y_true: 0 = 176296 | 1 = 251156
acc

In [14]:
"arch 1",'epoch 1000'
if __name__ == '__main__':
    print ('-----------------------------------this is train task--------------------------------')
    log_dir(args_global.train_config, args_global.data_prefix, git_branch, git_rev, timestamp)
    train_params, train_phases, arch_gcn = parse_n_prepare(args_global)
    print(train_params,train_phases,arch_gcn)
    tmp_data = data_prepare_modified(train_label,test_BindingDB_label, 'BindingDB',graph_1,feats_1,1)
    train_data = process_graph_data(*tmp_data)
    if 'eval_val_every' not in train_params:
        train_params['eval_val_every'] = 100
    model, minibatch, minibatch_eval, model_eval = prepare_1(train_data, train_params, arch_gcn)          
    train_1(train_phases, model, minibatch, minibatch_eval, model_eval, train_params['eval_val_every'])

-----------------------------------this is train task--------------------------------
{'lr': 0.001, 'weight_decay': 0.0, 'norm_loss': True, 'norm_aggr': True, 'q_threshold': 50, 'q_offset': 0, 'dropout': 0.1, 'sample_coverage': 50, 'positive_weight': 10} [{'end': 1000, 'sampler': 'edge', 'size_subg_edge': 2000, 'num_root': 750, 'depth': 4}] {'dim': 512, 'aggr': 'concat', 'loss': 'sigmoid', 'arch': '1', 'act': 'relu', 'bias': 'norm'}
task:BindingDB!!!!
训练集数量： 399553
测试集数量： 27899
427452
<class 'list'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
tr: 341960 va 85490 te 27899
全部link的数量： 427452.0   训练link的数量：  319631.0
features: <class 'numpy.ndarray'> (427452, 200)
TOTAL NUM OF PARAMS = 209926
dir_saver:  ./pytorch_models
path_saver:  ./pytorch_models/DTI_saved_model_2023-11-11 18-22-53.pkl
START PHASE    0
sampling 200 subgraphs:   time = 0.213 sec
0.0
0.0
Epoch    0
tn = 88091, fp = 88205, fn = 148035, tp = 103121
y_pred: 0 = 236126 | 1 = 191326
y_true: 0 = 176296 | 1 = 251156
acc=0.4

In [16]:
"arch 1-1",'epoch 1000'
if __name__ == '__main__':
    print ('-----------------------------------this is train task--------------------------------')
    log_dir(args_global.train_config, args_global.data_prefix, git_branch, git_rev, timestamp)
    train_params, train_phases, arch_gcn = parse_n_prepare(args_global)
    print(train_params,train_phases,arch_gcn)
    tmp_data = data_prepare_modified(train_label,test_BindingDB_label, 'BindingDB',graph_1,feats_1,1)
    train_data = process_graph_data(*tmp_data)
    if 'eval_val_every' not in train_params:
        train_params['eval_val_every'] = 100
    model, minibatch, minibatch_eval, model_eval = prepare_1(train_data, train_params, arch_gcn)          
    train_1(train_phases, model, minibatch, minibatch_eval, model_eval, train_params['eval_val_every'])

-----------------------------------this is train task--------------------------------
{'lr': 0.001, 'weight_decay': 0.0, 'norm_loss': True, 'norm_aggr': True, 'q_threshold': 50, 'q_offset': 0, 'dropout': 0.1, 'sample_coverage': 50, 'positive_weight': 10} [{'end': 1000, 'sampler': 'edge', 'size_subg_edge': 2000, 'num_root': 750, 'depth': 4}] {'dim': 512, 'aggr': 'concat', 'loss': 'sigmoid', 'arch': '1-1', 'act': 'relu', 'bias': 'norm'}
task:BindingDB!!!!
训练集数量： 399553
测试集数量： 27899
427452
<class 'list'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
tr: 341960 va 85490 te 27899
全部link的数量： 427452.0   训练link的数量：  319631.0
features: <class 'numpy.ndarray'> (427452, 200)
TOTAL NUM OF PARAMS = 1261574
dir_saver:  ./pytorch_models
path_saver:  ./pytorch_models/DTI_saved_model_2023-11-11 18-22-53.pkl
START PHASE    0
sampling 200 subgraphs:   time = 0.321 sec
0.0
0.0
Epoch    0
tn = 125120, fp = 51176, fn = 165626, tp = 85530
y_pred: 0 = 290746 | 1 = 136706
y_true: 0 = 176296 | 1 = 251156
acc=

In [17]:
"arch 1-0",'epoch 1000'
if __name__ == '__main__':
    print ('-----------------------------------this is train task--------------------------------')
    log_dir(args_global.train_config, args_global.data_prefix, git_branch, git_rev, timestamp)
    train_params, train_phases, arch_gcn = parse_n_prepare(args_global)
    print(train_params,train_phases,arch_gcn)
    tmp_data = data_prepare_modified(train_label,test_BindingDB_label, 'BindingDB',graph_1,feats_1,1)
    train_data = process_graph_data(*tmp_data)
    if 'eval_val_every' not in train_params:
        train_params['eval_val_every'] = 100
    model, minibatch, minibatch_eval, model_eval = prepare_1(train_data, train_params, arch_gcn)          
    train_1(train_phases, model, minibatch, minibatch_eval, model_eval, train_params['eval_val_every'])

-----------------------------------this is train task--------------------------------
{'lr': 0.001, 'weight_decay': 0.0, 'norm_loss': True, 'norm_aggr': True, 'q_threshold': 50, 'q_offset': 0, 'dropout': 0.1, 'sample_coverage': 50, 'positive_weight': 10} [{'end': 1000, 'sampler': 'edge', 'size_subg_edge': 2000, 'num_root': 750, 'depth': 4}] {'dim': 512, 'aggr': 'concat', 'loss': 'sigmoid', 'arch': '1-0', 'act': 'relu', 'bias': 'norm'}
task:BindingDB!!!!
训练集数量： 399553
测试集数量： 27899
427452
<class 'list'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
tr: 341960 va 85490 te 27899
全部link的数量： 427452.0   训练link的数量：  319631.0
features: <class 'numpy.ndarray'> (427452, 200)
TOTAL NUM OF PARAMS = 734726
dir_saver:  ./pytorch_models
path_saver:  ./pytorch_models/DTI_saved_model_2023-11-11 18-22-53.pkl
START PHASE    0
sampling 200 subgraphs:   time = 0.339 sec
0.0
0.0
Epoch    0
tn = 108224, fp = 68072, fn = 150266, tp = 100890
y_pred: 0 = 258490 | 1 = 168962
y_true: 0 = 176296 | 1 = 251156
acc=

In [18]:
"arch 1-0-1",'epoch 1000'
if __name__ == '__main__':
    print ('-----------------------------------this is train task--------------------------------')
    log_dir(args_global.train_config, args_global.data_prefix, git_branch, git_rev, timestamp)
    train_params, train_phases, arch_gcn = parse_n_prepare(args_global)
    print(train_params,train_phases,arch_gcn)
    tmp_data = data_prepare_modified(train_label,test_BindingDB_label, 'BindingDB',graph_1,feats_1,1)
    train_data = process_graph_data(*tmp_data)
    if 'eval_val_every' not in train_params:
        train_params['eval_val_every'] = 100
    model, minibatch, minibatch_eval, model_eval = prepare_1(train_data, train_params, arch_gcn)          
    train_1(train_phases, model, minibatch, minibatch_eval, model_eval, train_params['eval_val_every'])

-----------------------------------this is train task--------------------------------
{'lr': 0.001, 'weight_decay': 0.0, 'norm_loss': True, 'norm_aggr': True, 'q_threshold': 50, 'q_offset': 0, 'dropout': 0.1, 'sample_coverage': 50, 'positive_weight': 10} [{'end': 1000, 'sampler': 'edge', 'size_subg_edge': 2000, 'num_root': 750, 'depth': 4}] {'dim': 512, 'aggr': 'concat', 'loss': 'sigmoid', 'arch': '1-0-1', 'act': 'relu', 'bias': 'norm'}
task:BindingDB!!!!
训练集数量： 399553
测试集数量： 27899
427452
<class 'list'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
tr: 341960 va 85490 te 27899
全部link的数量： 427452.0   训练link的数量：  319631.0
features: <class 'numpy.ndarray'> (427452, 200)
TOTAL NUM OF PARAMS = 1263110
dir_saver:  ./pytorch_models
path_saver:  ./pytorch_models/DTI_saved_model_2023-11-11 18-22-53.pkl
START PHASE    0
sampling 200 subgraphs:   time = 0.329 sec
0.0
0.0
Epoch    0
tn = 92899, fp = 83397, fn = 130149, tp = 121007
y_pred: 0 = 223048 | 1 = 204404
y_true: 0 = 176296 | 1 = 251156
ac

In [19]:
"arch 1-1-1",'epoch 1000'
if __name__ == '__main__':
    print ('-----------------------------------this is train task--------------------------------')
    log_dir(args_global.train_config, args_global.data_prefix, git_branch, git_rev, timestamp)
    train_params, train_phases, arch_gcn = parse_n_prepare(args_global)
    print(train_params,train_phases,arch_gcn)
    tmp_data = data_prepare_modified(train_label,test_BindingDB_label, 'BindingDB',graph_1,feats_1,1)
    train_data = process_graph_data(*tmp_data)
    if 'eval_val_every' not in train_params:
        train_params['eval_val_every'] = 100
    model, minibatch, minibatch_eval, model_eval = prepare_1(train_data, train_params, arch_gcn)          
    train_1(train_phases, model, minibatch, minibatch_eval, model_eval, train_params['eval_val_every'])

-----------------------------------this is train task--------------------------------
{'lr': 0.001, 'weight_decay': 0.0, 'norm_loss': True, 'norm_aggr': True, 'q_threshold': 50, 'q_offset': 0, 'dropout': 0.1, 'sample_coverage': 50, 'positive_weight': 10} [{'end': 1000, 'sampler': 'edge', 'size_subg_edge': 2000, 'num_root': 750, 'depth': 4}] {'dim': 512, 'aggr': 'concat', 'loss': 'sigmoid', 'arch': '1-1-1', 'act': 'relu', 'bias': 'norm'}
task:BindingDB!!!!
训练集数量： 399553
测试集数量： 27899
427452
<class 'list'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
tr: 341960 va 85490 te 27899
全部link的数量： 427452.0   训练link的数量：  319631.0
features: <class 'numpy.ndarray'> (427452, 200)
TOTAL NUM OF PARAMS = 2313222
dir_saver:  ./pytorch_models
path_saver:  ./pytorch_models/DTI_saved_model_2023-11-11 18-22-53.pkl
START PHASE    0
sampling 200 subgraphs:   time = 0.340 sec
0.0
0.0
Epoch    0
tn = 144158, fp = 32138, fn = 209994, tp = 41162
y_pred: 0 = 354152 | 1 = 73300
y_true: 0 = 176296 | 1 = 251156
acc

In [22]:
"arch 0-0-0",'epoch 1000'
if __name__ == '__main__':
    print ('-----------------------------------this is train task--------------------------------')
    log_dir(args_global.train_config, args_global.data_prefix, git_branch, git_rev, timestamp)
    train_params, train_phases, arch_gcn = parse_n_prepare(args_global)
    print(train_params,train_phases,arch_gcn)
    tmp_data = data_prepare_modified(train_label,test_BindingDB_label, 'BindingDB',graph_1,feats_1,10)
    train_data = process_graph_data(*tmp_data)
    if 'eval_val_every' not in train_params:
        train_params['eval_val_every'] = 100
    model, minibatch, minibatch_eval, model_eval = prepare_1(train_data, train_params, arch_gcn)          
    train_1(train_phases, model, minibatch, minibatch_eval, model_eval, train_params['eval_val_every'])

-----------------------------------this is train task--------------------------------
{'lr': 0.001, 'weight_decay': 0.0, 'norm_loss': True, 'norm_aggr': True, 'q_threshold': 50, 'q_offset': 0, 'dropout': 0.1, 'sample_coverage': 50, 'positive_weight': 10} [{'end': 1000, 'sampler': 'full_batch', 'size_subg_edge': 2000, 'num_root': 750, 'depth': 4}] {'dim': 512, 'aggr': 'concat', 'loss': 'sigmoid', 'arch': '0-0-0', 'act': 'relu', 'bias': 'norm'}
task:BindingDB!!!!
训练集数量： 399553
测试集数量： 27899
427452
<class 'list'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
tr: 341960 va 85490 te 27899
全部link的数量： 427452.0   训练link的数量：  319745.0
features: <class 'numpy.ndarray'> (427452, 200)
TOTAL NUM OF PARAMS = 632326
dir_saver:  ./pytorch_models
path_saver:  ./pytorch_models/DTI_saved_model_2023-11-11 18-22-53.pkl
START PHASE    0
sampling 200 subgraphs:   time = 2.710 sec
0.0
4443200.0
Epoch    0
tn = 124757, fp = 51539, fn = 172530, tp = 78626
y_pred: 0 = 297287 | 1 = 130165
y_true: 0 = 176296 | 1 

In [25]:
"arch 1-0-0",'epoch 1000'
if __name__ == '__main__':
    print ('-----------------------------------this is train task--------------------------------')
    log_dir(args_global.train_config, args_global.data_prefix, git_branch, git_rev, timestamp)
    train_params, train_phases, arch_gcn = parse_n_prepare(args_global)
    print(train_params,train_phases,arch_gcn)
    tmp_data = data_prepare_modified(train_label,test_BindingDB_label, 'BindingDB',graph_1,feats_1,10)
    train_data = process_graph_data(*tmp_data)
    if 'eval_val_every' not in train_params:
        train_params['eval_val_every'] = 100
    model, minibatch, minibatch_eval, model_eval = prepare_1(train_data, train_params, arch_gcn)          
    train_1(train_phases, model, minibatch, minibatch_eval, model_eval, train_params['eval_val_every'])

-----------------------------------this is train task--------------------------------
{'lr': 0.001, 'weight_decay': 0.0, 'norm_loss': True, 'norm_aggr': True, 'q_threshold': 50, 'q_offset': 0, 'dropout': 0.1, 'sample_coverage': 50, 'positive_weight': 10} [{'end': 1000, 'sampler': 'full_batch', 'size_subg_edge': 2000, 'num_root': 750, 'depth': 4}] {'dim': 512, 'aggr': 'concat', 'loss': 'sigmoid', 'arch': '1-0-0', 'act': 'relu', 'bias': 'norm'}
task:BindingDB!!!!
训练集数量： 399553
测试集数量： 27899
427452
<class 'list'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
tr: 341960 va 85490 te 27899
全部link的数量： 427452.0   训练link的数量：  319745.0
features: <class 'numpy.ndarray'> (427452, 200)
TOTAL NUM OF PARAMS = 998406
dir_saver:  ./pytorch_models
path_saver:  ./pytorch_models/DTI_saved_model_2023-11-11 18-22-53.pkl
START PHASE    0
sampling 200 subgraphs:   time = 1.226 sec
0.0
4443200.0
Epoch    0
tn = 154459, fp = 21837, fn = 222737, tp = 28419
y_pred: 0 = 377196 | 1 = 50256
y_true: 0 = 176296 | 1 =

In [15]:
"arch 1-1-1-1",'epoch 1000'
if __name__ == '__main__':
    print ('-----------------------------------this is train task--------------------------------')
    log_dir(args_global.train_config, args_global.data_prefix, git_branch, git_rev, timestamp)
    train_params, train_phases, arch_gcn = parse_n_prepare(args_global)
    print(train_params,train_phases,arch_gcn)
    tmp_data = data_prepare_modified(train_label,test_BindingDB_label, 'BindingDB',graph_1,feats_1,0)
    train_data = process_graph_data(*tmp_data)
    if 'eval_val_every' not in train_params:
        train_params['eval_val_every'] = 100
    model, minibatch, minibatch_eval, model_eval = prepare_1(train_data, train_params, arch_gcn)          
    train_1(train_phases, model, minibatch, minibatch_eval, model_eval, train_params['eval_val_every'])

-----------------------------------this is train task--------------------------------
{'lr': 0.001, 'weight_decay': 0.0, 'norm_loss': True, 'norm_aggr': True, 'q_threshold': 50, 'q_offset': 0, 'dropout': 0.1, 'sample_coverage': 50, 'positive_weight': 10} [{'end': 1000, 'sampler': 'full_batch', 'size_subg_edge': 2000, 'num_root': 750, 'depth': 4}] {'dim': 512, 'aggr': 'concat', 'loss': 'sigmoid', 'arch': '1-1-1-1', 'act': 'relu', 'bias': 'norm'}
task:BindingDB!!!!
训练集数量： 399553
测试集数量： 27899
427452
<class 'list'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
tr: 341960 va 85490 te 27899
全部link的数量： 427452.0   训练link的数量：  319430.0
features: <class 'numpy.ndarray'> (427452, 200)
TOTAL NUM OF PARAMS = 3364870
dir_saver:  ./pytorch_models
path_saver:  ./pytorch_models/DTI_saved_model_2023-11-15 16-09-55.pkl
START PHASE    0
sampling 200 subgraphs:   time = 0.844 sec
0.0
4506200.0
Epoch    0
tn = 97967, fp = 78329, fn = 142387, tp = 108769
y_pred: 0 = 240354 | 1 = 187098
y_true: 0 = 176296 |

In [16]:
"arch 1-1-1",'epoch 1000'
if __name__ == '__main__':
    print ('-----------------------------------this is train task--------------------------------')
    log_dir(args_global.train_config, args_global.data_prefix, git_branch, git_rev, timestamp)
    train_params, train_phases, arch_gcn = parse_n_prepare(args_global)
    print(train_params,train_phases,arch_gcn)
    tmp_data = data_prepare_modified(train_label,test_BindingDB_label, 'BindingDB',graph_1,feats_1,1)
    train_data = process_graph_data(*tmp_data)
    if 'eval_val_every' not in train_params:
        train_params['eval_val_every'] = 100
    model, minibatch, minibatch_eval, model_eval = prepare_1(train_data, train_params, arch_gcn)          
    train_1(train_phases, model, minibatch, minibatch_eval, model_eval, train_params['eval_val_every'])

-----------------------------------this is train task--------------------------------
{'lr': 0.001, 'weight_decay': 0.0, 'norm_loss': True, 'norm_aggr': True, 'q_threshold': 50, 'q_offset': 0, 'dropout': 0.1, 'sample_coverage': 50, 'positive_weight': 10} [{'end': 1000, 'sampler': 'full_batch', 'size_subg_edge': 2000, 'num_root': 750, 'depth': 4}] {'dim': 512, 'aggr': 'concat', 'loss': 'sigmoid', 'arch': '1-1-1', 'act': 'relu', 'bias': 'norm'}
task:BindingDB!!!!
训练集数量： 399553
测试集数量： 27899
427452
<class 'list'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
tr: 341960 va 85490 te 27899
全部link的数量： 427452.0   训练link的数量：  319631.0
features: <class 'numpy.ndarray'> (427452, 200)
TOTAL NUM OF PARAMS = 2313222
dir_saver:  ./pytorch_models
path_saver:  ./pytorch_models/DTI_saved_model_2023-11-15 16-09-55.pkl
START PHASE    0
sampling 200 subgraphs:   time = 0.887 sec
0.0
4466000.0
Epoch    0
tn = 48505, fp = 127791, fn = 68981, tp = 182175
y_pred: 0 = 117486 | 1 = 309966
y_true: 0 = 176296 | 1